In [1]:
import requests
import pandas as pd
import json
from io import StringIO
from datetime import datetime, timedelta

In [18]:
def get_historical_data(code, start_date) :

    # 기업 인베스팅닷컴 고유 아이디를 구하기
    url = 'https://www.investing.com/search/service/search'
    headers = {
            'User-Agent':'Mozilla',
            'X-Requested-With':'XMLHttpRequest',
        }
    
    datas = {
        'search_text': code,
        'term': code,
        'country_id': '0',
        'tab_id': 'All'
        }
    res = requests.post(url, data=datas, headers=headers)
    
    
    # 해당 데이에서 curr_id 구하기
    code_dict = json.loads(res.text)
    
   
    curr_id = str(list(filter(lambda x : x['symbol'] == code, code_dict['All']))[0]['pair_ID'])
    
    today = datetime.strftime(datetime.utcnow() + timedelta(hours = 9), "%m/%d/%Y")
    
    
    # 주가가져오기

    url_data = 'https://www.investing.com/instruments/HistoricalDataAjax'
    
    datas = {    
                'curr_id': curr_id,   # 앞에서 구한 기업 고유 아이디
                'st_date': start_date,  # 함수에서 입력받은 날짜
                'end_date': today,   # 오늘날짜(변경가능)
                'interval_sec':'Daily',
                'sort_col':'date',
                'sort_ord':'ASC',
                'action':'historical_data'
            }

    headers = {
            'User-Agent':'Mozilla',
            'X-Requested-With':'XMLHttpRequest',
            }

    res_data = requests.post(url_data, data=datas, headers=headers)
    
    
    return pd.read_html(res_data.text)[0]    # 데이터 데이블 받기

In [19]:
start_date = '07/01/2019'
code = '005930'

price_table = get_historical_data(code, start_date) 

In [20]:
price_table['Date'] = pd.to_datetime(price_table['Date'])

In [21]:
price_r = price_table[::-1]

In [22]:
price_r.to_csv("sample_curprice.csv", mode='w', encoding='euckr')

PermissionError: [Errno 13] Permission denied: 'sample_curprice.csv'